In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

# Load pre-trained ResNet50 model
resnet50 = models.resnet50(pretrained=True)
# Remove the last classification layer
resnet50 = torch.nn.Sequential(*(list(resnet50.children())[:-1]))

# Define ImageNet normalization parameters
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Define transforms to resize, normalize, and convert images to tensors
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load images of cats and dogs
cat_images = []
dog_images = []
for i in range(1, 4):
    cat_img = Image.open(f'cat-{i}.jpeg')
    dog_img = Image.open(f'dog-{i}.jpeg')
    cat_images.append(transform(cat_img))
    dog_images.append(transform(dog_img))

# Extract features for cat and dog images
def extract_features(images):
    features = []
    for img in images:
        with torch.no_grad():
            img = img.unsqueeze(0)
            feature = resnet50(img)
            feature = feature.squeeze().numpy()
            features.append(feature)
    return features

cat_features = extract_features(cat_images)
dog_features = extract_features(dog_images)

# Compute cosine distance and L2 norm
def cosine_distance(u, v):
    return 1 - np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

def l2_norm(u, v):
    return np.linalg.norm(u - v)

# Compare cat features with each other
print("Cosine distance between cat features:")
for i in range(len(cat_features)):
    for j in range(i + 1, len(cat_features)):
        dist = cosine_distance(cat_features[i], cat_features[j])
        print(f"Cat {i+1} vs Cat {j+1}: {dist}")

print("\nL2 norm between cat features:")
for i in range(len(cat_features)):
    for j in range(i + 1, len(cat_features)):
        dist = l2_norm(cat_features[i], cat_features[j])
        print(f"Cat {i+1} vs Cat {j+1}: {dist}")

# Compare dog features with each other
print("\nCosine distance between dog features:")
for i in range(len(dog_features)):
    for j in range(i + 1, len(dog_features)):
        dist = cosine_distance(dog_features[i], dog_features[j])
        print(f"Dog {i+1} vs Dog {j+1}: {dist}")

print("\nL2 norm between dog features:")
for i in range(len(dog_features)):
    for j in range(i + 1, len(dog_features)):
        dist = l2_norm(dog_features[i], dog_features[j])
        print(f"Dog {i+1} vs Dog {j+1}: {dist}")


C:\Users\riori\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\riori\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\riori\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\riori\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\riori\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most u

Cosine distance between cat features:
Cat 1 vs Cat 2: 0.0042920708656311035
Cat 1 vs Cat 3: 0.004217803478240967
Cat 2 vs Cat 3: 0.004291892051696777

L2 norm between cat features:
Cat 1 vs Cat 2: 2.020491361618042
Cat 1 vs Cat 3: 1.9662295579910278
Cat 2 vs Cat 3: 2.0051419734954834

Cosine distance between dog features:
Dog 1 vs Dog 2: 0.0041792988777160645
Dog 1 vs Dog 3: 0.00434952974319458
Dog 2 vs Dog 3: 0.004631519317626953

L2 norm between dog features:
Dog 1 vs Dog 2: 1.9651762247085571
Dog 1 vs Dog 3: 2.0222537517547607
Dog 2 vs Dog 3: 2.1005399227142334


In [2]:
# Compare cat features with dog features
print("\nCosine distance between cat and dog features:")
for i in range(len(cat_features)):
    for j in range(len(dog_features)):
        dist = cosine_distance(cat_features[i], dog_features[j])
        print(f"Cat {i+1} vs Dog {j+1}: {dist}")

print("\nL2 norm between cat and dog features:")
for i in range(len(cat_features)):
    for j in range(len(dog_features)):
        dist = l2_norm(cat_features[i], dog_features[j])
        print(f"Cat {i+1} vs Dog {j+1}: {dist}")



Cosine distance between cat and dog features:
Cat 1 vs Dog 1: 0.004583597183227539
Cat 1 vs Dog 2: 0.004755914211273193
Cat 1 vs Dog 3: 0.004564762115478516
Cat 2 vs Dog 1: 0.004246354103088379
Cat 2 vs Dog 2: 0.004286289215087891
Cat 2 vs Dog 3: 0.004675447940826416
Cat 3 vs Dog 1: 0.004382789134979248
Cat 3 vs Dog 2: 0.004277467727661133
Cat 3 vs Dog 3: 0.004632771015167236

L2 norm between cat and dog features:
Cat 1 vs Dog 1: 2.060588836669922
Cat 1 vs Dog 2: 2.0776333808898926
Cat 1 vs Dog 3: 2.0945916175842285
Cat 2 vs Dog 1: 1.9914679527282715
Cat 2 vs Dog 2: 2.0115714073181152
Cat 2 vs Dog 3: 2.0948598384857178
Cat 3 vs Dog 1: 2.0105555057525635
Cat 3 vs Dog 2: 1.9789475202560425
Cat 3 vs Dog 3: 2.091280221939087


In [7]:
import random

class MemoryTrace:
    def __init__(self, feature, strength):
        self.feature = feature
        self.strength = strength

class SensoryRegister:
    def __init__(self, max_size=100):
        self.max_size = max_size
        self.queue = []

    def push(self, memory_trace):
        if len(self.queue) >= self.max_size:
            self.queue.pop(0)  # Remove oldest memory trace
        self.queue.append(memory_trace)

    def pop(self):
        if len(self.queue) == 0:
            return None
        return self.queue.pop(0)

class ShortTermStore:
    def __init__(self, max_capacity=7, decay_constant=0.05):
        self.max_capacity = max_capacity
        self.decay_constant = decay_constant
        self.queue = []

    def push(self, memory_trace):
        if len(self.queue) >= self.max_capacity:
            # Remove memory trace with lowest strength
            self.queue.sort(key=lambda x: x.strength)
            self.queue.pop(0)
        self.queue.append(memory_trace)

    def decay(self):
        for trace in self.queue:
            trace.strength -= self.decay_constant
            if trace.strength <= 0:
                self.queue.remove(trace)


class LongTermStore:
    def __init__(self, decay_constant=0.0005):
        self.decay_constant = decay_constant
        self.memory_traces = []

    def push(self, memory_trace):
        self.memory_traces.append(memory_trace)

    def decay(self):
        for trace in self.memory_traces:
            trace.strength -= self.decay_constant
            if trace.strength <= 0:
                self.memory_traces.remove(trace)

def information_decay(feature, decay_constant):
    length = len(feature)
    n = int(length * decay_constant)
    idxes = random.sample(range(length), n)
    for i in idxes:
        feature[i] = random.uniform(0, 1)
    return feature

# Usage example
# Assume cat_features and dog_features are lists of MemoryTrace objects

# Simulate decay in short-term store
short_term_store = ShortTermStore()
for trace in cat_features + dog_features:
    short_term_store.push(trace)

short_term_store.decay()  # Simulate decay

# Simulate decay in long-term store
long_term_store = LongTermStore()
for trace in cat_features + dog_features:
    long_term_store.push(trace)

long_term_store.decay()  # Simulate decay


AttributeError: 'numpy.ndarray' object has no attribute 'strength'

In [8]:
import numpy as np
import random

class MemoryTrace:
    def __init__(self, feature, strength):
        self.feature = feature
        self.strength = strength

class ShortTermStore:
    def __init__(self, max_capacity=7, decay_constant=0.05):
        self.max_capacity = max_capacity
        self.decay_constant = decay_constant
        self.queue = []

    def push(self, memory_trace):
        if len(self.queue) >= self.max_capacity:
            # Remove memory trace with lowest strength
            self.queue.sort(key=lambda x: x.strength)
            self.queue.pop(0)
        self.queue.append(memory_trace)

    def decay(self):
        for trace in self.queue:
            trace.strength -= self.decay_constant
            if trace.strength <= 0:
                self.queue.remove(trace)

class LongTermStore:
    def __init__(self, decay_constant=0.0005):
        self.decay_constant = decay_constant
        self.memory_traces = []

    def add_memory_trace(self, memory_trace):
        self.memory_traces.append(memory_trace)

    def decay(self):
        for trace in self.memory_traces:
            trace.strength -= self.decay_constant
            if trace.strength <= 0:
                self.memory_traces.remove(trace)

class SensoryRegister:
    def __init__(self, max_capacity=100):
        self.max_capacity = max_capacity
        self.queue = []

    def perceive(self, sensory_info):
        if len(self.queue) >= self.max_capacity:
            self.queue.pop(0)
        self.queue.append(sensory_info)

# Create feature vectors (just placeholders for demonstration)
cat_features = [np.random.rand(2048) for _ in range(6)]
dog_features = [np.random.rand(2048) for _ in range(6)]

# Create memory traces with random strengths
cat_traces = [MemoryTrace(feature, random.uniform(0.5, 1.0)) for feature in cat_features]
dog_traces = [MemoryTrace(feature, random.uniform(0.5, 1.0)) for feature in dog_features]

# Create short-term store
short_term_store = ShortTermStore()

# Push memory traces to short-term store
for trace in cat_traces + dog_traces:
    short_term_store.push(trace)

short_term_store.decay()  # Simulate decay

# Create long-term store
long_term_store = LongTermStore()

# Add memory traces to long-term store
for trace in cat_traces + dog_traces:
    long_term_store.add_memory_trace(trace)

long_term_store.decay()  # Simulate decay

# Create sensory register
sensory_register = SensoryRegister()

# Perceive sensory information
sensory_info = np.random.rand(100)  # Placeholder for sensory information
sensory_register.perceive(sensory_info)


In [10]:
import torch
import torchvision.models as models

class ImageRecovery:
    def __init__(self):
        # Load i-RevNet model
        self.model = models.iRevNet(num_classes=1000, layers=[4, 6, 8, 10], final_stride=2)
        self.model.eval()  # Set model to evaluation mode

    def recover_image(self, features):
        # Flatten features
        features_flat = features.flatten()
        
        # Reshape features to [6, 3072, 7, 7]
        features_reshaped = features_flat.reshape((6, 3072, 7, 7))
        
        # Convert to tensor and normalize
        features_tensor = torch.tensor(features_reshaped, dtype=torch.float32)
        features_tensor /= 255.0  # Assuming features are in [0, 255] range
        
        # Recover images
        with torch.no_grad():
            recovered_images = self.model(features_tensor)
        
        return recovered_images.numpy()

# Example usage
if __name__ == "__main__":
    # Assume you have a 'features' variable containing the encoded features
    # Initialize ImageRecovery class
    image_recovery = ImageRecovery()

    # Recover images from features
    recovered_images = image_recovery.recover_image(features)


AttributeError: module 'torchvision.models' has no attribute 'iRevNet'

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# Define the SplittingOperator module
class SplittingOperator(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SplittingOperator, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        return self.conv(x)

# Define the DownsamplingOperator module
class DownsamplingOperator(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DownsamplingOperator, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        return self.conv(x)

# Define the BottleneckBlock module
class BottleneckBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(BottleneckBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels // 4, kernel_size=1, stride=1, padding=0)
        self.conv2 = nn.Conv2d(out_channels // 4, out_channels // 4, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(out_channels // 4, out_channels, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(out_channels // 4)
        self.bn2 = nn.BatchNorm2d(out_channels // 4)
        self.bn3 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        return out

# Define the iRevNet module
class IRevNet(nn.Module):
    def __init__(self, num_layers, num_blocks, num_channels):
        super(IRevNet, self).__init__()
        self.num_layers = num_layers
        self.num_blocks = num_blocks
        self.num_channels = num_channels

        self.splitting_operator = SplittingOperator(3, self.num_channels[0])

        self.blocks = nn.ModuleList()
        for i in range(num_blocks):
            self.blocks.append(BottleneckBlock(self.num_channels[i % len(self.num_channels)], self.num_channels[i % len(self.num_channels)] * 4))

        self.downsampling_operators = nn.ModuleList()
        for i in range(1, num_layers):  # Limit loop iterations till layer 3 or num_layers (whichever is smaller)
            if i == 3 or i == 21 or i == 69 or i == 285:
                self.downsampling_operators.append(DownsamplingOperator(self.num_channels[i], self.num_channels[i]))

    def forward(self, x):
        out = self.splitting_operator(x)
        for i in range(self.num_blocks):
            out = self.blocks[i](out)
            if i + 1 in [3, 21, 69, 285]:
                out = self.downsampling_operators[i // 3](out)
        return out

# Define the Decay Simulation module
class DecaySimulation:
    def __init__(self, decay_percentage):
        self.decay_percentage = decay_percentage

    def decay_feature(self, feature):
        num_decay_elements = int(feature.size(0) * self.decay_percentage)
        decay_indices = np.random.choice(feature.size(0), num_decay_elements, replace=False)
        feature[decay_indices] = 0
        return feature


# Define the Sensory Register module
class SensoryRegister:
    def __init__(self, max_capacity):
        self.max_capacity = max_capacity
        self.queue = []

    def push(self, feature):
        if len(self.queue) < self.max_capacity:
            self.queue.append(feature)
        else:
            self.queue.pop(0)
            self.queue.append(feature)

    def pop(self):
        if len(self.queue) > 0:
            return self.queue.pop(0)
        else:
            return None

# Define the Short-term Store module
class ShortTermStore:
    def __init__(self, max_capacity):
        self.max_capacity = max_capacity
        self.queue = []

    def push(self, feature):
        if len(self.queue) < self.max_capacity:
            self.queue.append(feature)
        else:
            self.queue.pop(0)
            self.queue.append(feature)

    def pop(self):
        if len(self.queue) > 0:
            return self.queue.pop(0)
        else:
            return None

    def decay(self):
        # Simulate decay process
        for feature in self.queue:
            # Decay the strength of each memory trace
            feature -= 0.05  # Adjust the decay constant as needed
            feature[feature < 0] = 0  # Ensure strength doesn't go negative

    def recall(self, feature):
        # Locate the closest memory trace and return
        closest_feature = None
        min_distance = float('inf')
        for stored_feature in self.queue:
            distance = torch.dist(feature, stored_feature)
            if distance < min_distance:
                min_distance = distance
                closest_feature = stored_feature
        return closest_feature

# Define the Long-term Store module
class LongTermStore:
    def __init__(self):
        self.memory_traces = []

    def push(self, feature):
        self.memory_traces.append(feature)

    def pop(self):
        if len(self.memory_traces) > 0:
            return self.memory_traces.pop(0)
        else:
            return None

    def decay(self):
        # Simulate decay process
        for feature in self.memory_traces:
            # Decay the strength of each memory trace
            feature -= 0.0005  # Adjust the decay constant as needed
            feature[feature < 0] = 0  # Ensure strength doesn't go negative

    def recall(self, feature):
        # Locate the closest memory trace and return
        closest_feature = None
        min_distance = float('inf')
        for stored_feature in self.memory_traces:
            distance = torch.dist(feature, stored_feature)
            if distance < min_distance:
                min_distance = distance
                closest_feature = stored_feature
        return closest_feature

# Reconstruction process
class ImageRecovery:
    def __init__(self, irevnet):
        self.irevnet = irevnet

    def recover_image(self, feature):
        # Apply inverse iRevNet to recover the image
        recovered_image = self.irevnet(feature)
        return recovered_image

# Initialize components
injective_i_revnet = IRevNet(num_layers=300, num_blocks=100, num_channels=[48, 192, 768, 3072])
decay_simulator = DecaySimulation(decay_percentage=0.1)
sensory_register = SensoryRegister(max_capacity=100)
short_term_store = ShortTermStore(max_capacity=7)
long_term_store = LongTermStore()

# Assuming cat features are provided
cat_features = [torch.randn(2048) for _ in range(3)]

# Assuming dog features are provided
dog_features = [torch.randn(2048) for _ in range(3)]

# Simulate processing cat features
for cat_feature in cat_features:
    cat_feature_decayed = decay_simulator.decay_feature(cat_feature)
    sensory_register.push(cat_feature_decayed)

# Simulate processing dog features
for dog_feature in dog_features:
    dog_feature_decayed = decay_simulator.decay_feature(dog_feature)
    sensory_register.push(dog_feature_decayed)

# Retrieve features from sensory register
while sensory_register.queue:
    feature = sensory_register.pop()
    short_term_store.push(feature)

# Simulate decay in short-term store
short_term_store.pop()
short_term_store.pop()
short_term_store.decay()

# Simulate decay in long-term store
long_term_store.decay()

# Reconstruction process
image_recovery = ImageRecovery(injective_i_revnet)
reconstructed_images = []
for feature in short_term_store.queue + long_term_store.memory_traces:
    recovered_image = image_recovery.recover_image(feature)
    reconstructed_images.append(recovered_image)

# Display the reconstructed images
for i, image in enumerate(reconstructed_images):
    print(f"Reconstructed Image {i + 1}: {image.shape}")  # Replace this with your display code


IndexError: list index out of range